## ECS 289G - Finetuned T5 Model for Grammar Error Correction

This notebook contains the project code for our Finetuned and Retrained T5 model for grammar error correction

In [ ]:
!pip install happytransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.5 MB/s eta 0:00:00


In [ ]:
!pip install nltk errant jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.3/499.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 22.5 MB/s eta 0:00:00


In [ ]:
#Here we are importing the T5 Text to Text Transfer Transformer from the Happy Transformer Library (Pretrained for General Machine Translation)

from happytransformer import HappyTextToText

happy_tt = HappyTextToText("T5", "t5-base")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
# The dataset for finetuning and retraining the T5 model for Grammar Error Correction is JFleg ( Sentence+Correction Pairs)

from datasets import load_dataset

train_dataset = load_dataset("jfleg", split='validation[:]')

eval_dataset = load_dataset("jfleg", split='test[:]')

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/748 [00:00<?, ? examples/s]

In [ ]:
for case in train_dataset["corrections"][:2]:
  print(case)
  print(case[0])
  print("--------------------------------------------------------")

['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . ']
So I think we would not be alive if our ancestors did not develop sciences and technologies . 
--------------------------------------------------------
['Not for use with a car . ', 'Do not use in the car . ', 'Car not for use . ', 'Can not use the car . ']
Not for use with a car . 
--------------------------------------------------------


In [ ]:
#Here is the script to convert the JFleg dataset into train.csv and eval.csv containing two columns - sentence and their corresponding corrections. This converted format has been

import csv

def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
     	    # Adding the task's prefix to input
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                # a few of the cases contain blank strings.
                if input_text and correction:
                    writter.writerow([input_text, correction])

#This converted format has been used in all three notebooks for grammar correction.

generate_csv("train.csv", train_dataset)
generate_csv("eval.csv", eval_dataset)


In [ ]:
before_result = happy_tt.eval("eval.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
#Evaluating the performance of the pretrained T5 model on Jfelg without any changes from our side

print("Before loss:", before_result.loss)

Before loss: 1.28038489818573


In [ ]:
from happytransformer import TTTrainArgs


#Here we finetune and retrain the model on our JFleg dataset to customize its learning for grammar error correction tasks
args = TTTrainArgs(batch_size=8)
happy_tt.train("train.csv", args=args)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2714 [00:00<?, ? examples/s]

Map:   0%|          | 0/302 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
1,1.555600,1.361047
34,0.849600,0.679001
68,0.752400,0.580555
102,0.662800,0.542612
136,0.637200,0.531399
170,0.628400,0.518125
204,0.612000,0.505470
238,0.602400,0.498149
272,0.570500,0.495834
306,0.548200,0.494440


In [ ]:
#Now we analyse the loss after our finetuning and see that we have a better loss
#The variable name is misleading as I have used before_loss but it is after loss as the model has been retrained)

before_loss = happy_tt.eval("eval.csv")

print("After loss: ", before_loss.loss)

Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

After loss:  1.2803850173950195


In [ ]:
from happytransformer import TTSettings

beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=20)

In [ ]:
happy_tt.save("pretrained.h5")

In [ ]:
import pandas as pd
eval_df = pd.read_csv("eval.csv")
eval_subset = eval_df

ground_truth_eval = eval_subset["target"].tolist()
predicted_eval = []

for input_text in eval_subset["input"].tolist():
    result = happy_tt.generate_text(input_text, args=beam_settings)
    predicted_eval.append(result.text)


In [ ]:
from nltk.translate import bleu_score
from nltk.translate.gleu_score import sentence_gleu
from jiwer import wer
from sklearn.metrics import precision_recall_fscore_support

def calculate_metrics(reference_list, predicted_list):
    # BLEU score
    bleu_scores = [bleu_score.sentence_bleu([ref.split()], pred.split()) for ref, pred in zip(reference_list, predicted_list)]
    bleu = sum(bleu_scores) / len(bleu_scores)

    # GLEU score
    gleu_scores = [sentence_gleu([ref.split()], pred.split()) for ref, pred in zip(reference_list, predicted_list)]
    gleu = sum(gleu_scores) / len(gleu_scores)

    # WER (Word Error Rate)
    wer_scores = [wer(ref, pred) for ref, pred in zip(reference_list, predicted_list)]
    wer_score = sum(wer_scores) / len(wer_scores)

    return bleu, gleu, wer_score


# Calculate metrics for the entire evaluation set
bleu_eval, gleu_eval, wer_eval = calculate_metrics(ground_truth_eval, predicted_eval)


# Print the overall results
print("Overall BLEU (eval):", bleu_eval)
print("Overall GLEU (eval):", gleu_eval)
print("Overall WER (eval):", wer_eval)


Overall BLEU (eval): 0.3339397489405879
Overall GLEU (eval): 0.4071529776101529
Overall WER (eval): 0.5095848965848966
